In [12]:
import pandas as pd

import numpy as np

# import sqlalchemy

# ============= USER INPUT  =============

In [11]:
%cd C:\ALL_DATA\URGummitha\Desktop\Post_Mod-2\sol103_nmodes

C:\ALL_DATA\URGummitha\Desktop\Post_Mod-2\sol103_nmodes


### LOADING GRID POINT DATA FOR DFEM & GFEM [Created by Hypermesh]

In [89]:
dfem_nodes_csv_file = 'Fuse_RBE3Nodes.csv'

gfem_nodes_csv_file = 'Fuse_RBE3Nodes.csv'

# DFEM HDF5 [Input file name]

In [90]:
h5_file = './sol101_limit/pc12-47-v1-6-static_post-mod_m2.51_limit.h5'

# GFEM HDF5 [Input file name]

In [91]:
GFEM_h5_file = 'pc12-47-v1-6-static_post-mod_m3.1_limit.h5'

# =============== END OF USER INPUT ===============

# Loading HDF5 FILES

In [92]:
store = pd.HDFStore(h5_file, mode='r')

In [93]:
gfem_store = pd.HDFStore(GFEM_h5_file, mode='r')

In [94]:
dfem_wing_rbe3_nodes = pd.read_csv(dfem_nodes_csv_file)

gfem_wing_rbe3_nodes = pd.read_csv(gfem_nodes_csv_file)

In [95]:
def get_df(x, eid_array, cols = list('AB')):
    
    df = pd.DataFrame(x[:,[0,10]],index=eid_array, columns=cols)
    
    return df

In [96]:
def Join_DF(df1,df2):
    
    if df1.empty == True:
        
        df1 = df2.copy()
        
    else :
        
        df1 = pd.concat([df1, df2], axis=1, sort=False, join_axes=[df1.index])
        
    return df1

In [97]:
def CBAR_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BAR')
    
    return df

In [98]:
def CBEAM_ELEMENTAL_FORCES(store):
    
    beam_node = store.get_node('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BEAM')

    beam_lst_names = beam_node.colnames[1:-1]

    res_df = pd.DataFrame({'A' : []})

    for x in beam_lst_names:

        col_name_a = x + '_A'

        col_name_b = x + '_B'

        df = get_df(beam_node.col(x), beam_node.col('EID'), cols=[col_name_a, col_name_b])

        res_df = Join_DF(res_df, df)
        
    domain_id_df = pd.DataFrame(beam_node.col('DOMAIN_ID'), index=beam_node.col('EID'), columns=['DOMAIN_ID'])
    
    res_df = Join_DF(res_df, domain_id_df)
    
    res_df.index.name = 'EID'
    
    return res_df

In [99]:
def CELAS2_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ELAS2')
    
    return df

In [100]:
def CQUAD4_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/QUAD4')
    
    return df

In [101]:
def CROD_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ROD')
    
    return df

In [102]:
def CBUSH_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BUSH')
    
    return df


In [103]:
def CSHEAR_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/SHEAR')
    
    return df

In [104]:
def CTRIA3_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/TRIA3')
    
    return df

In [105]:
def CBAR_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/BAR')
    
    return df

In [106]:
def CBEAM_ELEMENTAL_STRESS(store):
    
    beam_node = store.get_node('/NASTRAN/RESULT/ELEMENTAL/STRESS/BEAM')

    beam_lst_names = beam_node.colnames[1:-1]

    res_df = pd.DataFrame({'A' : []})

    for x in beam_lst_names:

        col_name_a = x + '_A'

        col_name_b = x + '_B'

        df = get_df(beam_node.col(x), beam_node.col('EID'), cols=[col_name_a, col_name_b])

        res_df = Join_DF(res_df, df)
        
    domain_id_df = pd.DataFrame(beam_node.col('DOMAIN_ID'), index=beam_node.col('EID'), columns=['DOMAIN_ID'])
    
    res_df.index.name = 'EID'
    
    res_df = Join_DF(res_df, domain_id_df)
    
    return res_df

In [107]:
def CELAS2_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/ELAS2')
    
    return df

In [108]:
def CQUAD4_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/QUAD4')
    
    return df

In [109]:
def CROD_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/ROD')
    
    return df

In [110]:
def CSHEAR_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/SHEAR')
    
    return df

In [111]:
def CTRIA3_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/TRIA3')
    
    return df

# DOMAIN_ID Mapping to SUBCASE

In [112]:
domain_results  =  store.get('/NASTRAN/RESULT/DOMAINS')

domain_results = domain_results.loc[:, ['ID', 'SUBCASE']].copy()

domain_results.set_index('ID', inplace=True)

subcase_mapping = domain_results.to_dict()

# subcase_mapping['SUBCASE']

# GRID POINT EXTRACTION

In [113]:
def GP_NODAL_DISPLACEMENT(store):
    
    df = store.get('/NASTRAN/RESULT/NODAL/DISPLACEMENT')
    
    return df

In [114]:
all_gp_dispacements = GP_NODAL_DISPLACEMENT(store)

# gp_dispacements.head()

### custom function to extract grid point forces for selected nodes.

In [115]:
def Custom_GP_Forces(all_gp_df, wing_gp_nodes_df ):

    filter_bool = all_gp_df.ID.isin(wing_gp_nodes_df.dependentnode.tolist())

    gp_dispacements = all_gp_df[filter_bool == True].copy()

    # gp_dispacements.head()

    gp_dispacements['SUBCASE'] = gp_dispacements.DOMAIN_ID.map(subcase_mapping['SUBCASE']).copy()

    # gp_dispacements.head()

    take_slice = wing_gp_nodes_df[['dependentnode', 'globalx', 'globaly', 'globalz']].copy()

    take_slice.columns = ['ID', 'X_Cord', 'Y_Cord', 'Z_Cord']

    # take_slice.head()

    merged_df  = gp_dispacements.merge(take_slice, on=['ID'], how='outer')

    # merged_df.head()

    merged_df.columns = ['DEPENDENT_NODE', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'DOMAIN_ID', 'SUBCASE', 'X_Cord',
           'Y_Cord', 'Z_Cord']

    # merged_df.head()

    merged_df['Rx_in_Degree'] = merged_df.RX * (180.0/np.pi)

    merged_df['Ry_in_Degree'] = merged_df.RY * (180.0/np.pi)

    merged_df['Rz_in_Degree'] = merged_df.RZ * (180.0/np.pi)
    
    # merged_df.to_csv('DFEM_wing_displacements.csv')
    
    return merged_df

In [116]:
def Custom_Switch_X_to_Y(df):
    
    new_column_order = ['DEPENDENT_NODE', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'DOMAIN_ID',
       'SUBCASE', 'Y_Cord', 'X_Cord', 'Z_Cord', 'Rx_in_Degree', 'Ry_in_Degree',
       'Rz_in_Degree']
    
    df = df.reindex(columns=new_column_order).copy()
    
    return df

In [117]:
# dfem_wing_rbe3_nodes = dfem_wing_rbe3_nodes[(dfem_wing_rbe3_nodes[['globalx', 'globaly', 'globalz']] != 0).all(axis=1)].copy()

In [118]:
# dfem_wing_rbe3_nodes.head()

In [119]:
dfem_extraction_df = Custom_GP_Forces(all_gp_dispacements, dfem_wing_rbe3_nodes)

dfem_extraction_df = Custom_Switch_X_to_Y(dfem_extraction_df)

# dfem_extraction_df.head()

In [120]:
# df = pd.DataFrame({'a':np.random.randn(3), 'b': np.random.randn(3), 'c':np.random.randn(3)})

# df.head()

In [121]:
# df = df.reindex(columns=['c','a','b'])

# df.head()

In [122]:
all_gfem_gp_displacemnts = GP_NODAL_DISPLACEMENT(gfem_store)

In [123]:
gfem_extraction_df = Custom_GP_Forces(all_gfem_gp_displacemnts, gfem_wing_rbe3_nodes)

gfem_extraction_df = Custom_Switch_X_to_Y(gfem_extraction_df)

# gfem_extraction_df.head()

In [124]:
with pd.ExcelWriter('FUSE_NODE_DISPLACEMENT_EXTRACTION.xlsx') as writer:
    
    dfem_extraction_df.to_excel(writer, sheet_name='DFEM_EXTRACTION')
    
    gfem_extraction_df.to_excel(writer, sheet_name='GFEM_EXTRACTION')

In [125]:
merged_df = dfem_extraction_df.merge(gfem_extraction_df, on=['DEPENDENT_NODE'], suffixes=('_DFEM', '_GFEM'))

In [126]:
# merged_df.head()

In [127]:
with pd.ExcelWriter('FUSE_NODE_DISPLACEMENT_EXTRACTION_COMBINED.xlsx') as writer:
    
    merged_df.to_excel(writer, sheet_name='COMBINED_DGFEM_EXTRACTION')

# Close the Entire Store HDF5

In [128]:
# closing the dfem store
store.close()

# closing the gfem store
gfem_store.close()

In [129]:
print('END OF PYTHON RUN.')

END OF PYTHON RUN.


# SAMPLE WORK

In [13]:
m28_model = pd.HDFStore('pc12-47-v1-6-static_post-mod_m2.48_nmodes.h5')


In [14]:
m51_model = pd.HDFStore('pc12-47-v1-6-static_post-mod_m2.51_limit.h5')

In [16]:
m28_model.keys()

['/NASTRAN/INPUT/DOMAINS',
 '/NASTRAN/RESULT/DOMAINS',
 '/NASTRAN/RESULT/NODAL/EIGENVECTOR',
 '/NASTRAN/RESULT/NODAL/GRID_WEIGHT',
 '/NASTRAN/RESULT/SUMMARY/EIGENVALUE',
 '/NASTRAN/INPUT/CONSTRAINT/SPC',
 '/NASTRAN/INPUT/CONSTRAINT/SUPORT',
 '/NASTRAN/INPUT/COORDINATE_SYSTEM/CORD2C',
 '/NASTRAN/INPUT/COORDINATE_SYSTEM/CORD2R',
 '/NASTRAN/INPUT/COORDINATE_SYSTEM/CORD2S',
 '/NASTRAN/INPUT/ELEMENT/CBAR',
 '/NASTRAN/INPUT/ELEMENT/CBEAM',
 '/NASTRAN/INPUT/ELEMENT/CBUSH',
 '/NASTRAN/INPUT/ELEMENT/CELAS2',
 '/NASTRAN/INPUT/ELEMENT/CHEXA',
 '/NASTRAN/INPUT/ELEMENT/CPENTA',
 '/NASTRAN/INPUT/ELEMENT/CQUAD4',
 '/NASTRAN/INPUT/ELEMENT/CROD',
 '/NASTRAN/INPUT/ELEMENT/CSHEAR',
 '/NASTRAN/INPUT/ELEMENT/CTETRA',
 '/NASTRAN/INPUT/ELEMENT/CTRIA3',
 '/NASTRAN/INPUT/ELEMENT/PLOTEL',
 '/NASTRAN/INPUT/ELEMENT/RBAR',
 '/NASTRAN/INPUT/LOAD/FORCE',
 '/NASTRAN/INPUT/LOAD/MOMENT',
 '/NASTRAN/INPUT/LOAD/PLOAD2',
 '/NASTRAN/INPUT/LOAD/PLOAD4',
 '/NASTRAN/INPUT/MATERIAL/MAT1',
 '/NASTRAN/INPUT/MATERIAL/MAT4',
 '/NA

In [17]:
RBE2_GM = '/NASTRAN/INPUT/ELEMENT/RBE2/GM'
RBE2_RB = '/NASTRAN/INPUT/ELEMENT/RBE2/RB'
RBE3_G = '/NASTRAN/INPUT/ELEMENT/RBE3/G'
RBE3_IDENTITY = '/NASTRAN/INPUT/ELEMENT/RBE3/IDENTITY'
RBE3_WTCG = '/NASTRAN/INPUT/ELEMENT/RBE3/WTCG'

In [19]:
def extract_df(model, path):
    df = model.get(path)
    return df

In [20]:
m28_RBE2_GM_df = extract_df(m28_model,'/NASTRAN/INPUT/ELEMENT/RBE2/GM')
m28_RBE2_RB_df = extract_df(m28_model,'/NASTRAN/INPUT/ELEMENT/RBE2/RB')
m28_RBE3_G_df = extract_df(m28_model,'/NASTRAN/INPUT/ELEMENT/RBE3/G')
m28_RBE3_IDENTITY_df = extract_df(m28_model,'/NASTRAN/INPUT/ELEMENT/RBE3/IDENTITY')
m28_RBE3_WTCG_df = extract_df(m28_model,'/NASTRAN/INPUT/ELEMENT/RBE3/WTCG')

In [29]:
print(m28_RBE2_GM_df.shape)
print(m28_RBE2_RB_df.shape)
print(m28_RBE3_G_df.shape)
print(m28_RBE3_IDENTITY_df.shape)
print(m28_RBE3_WTCG_df.shape)

(2633, 1)
(105, 7)
(612621, 1)
(118679, 9)
(582511, 4)


In [23]:
m51_RBE2_GM_df = extract_df(m51_model,'/NASTRAN/INPUT/ELEMENT/RBE2/GM')
m51_RBE2_RB_df = extract_df(m51_model,'/NASTRAN/INPUT/ELEMENT/RBE2/RB')
m51_RBE3_G_df = extract_df(m51_model,'/NASTRAN/INPUT/ELEMENT/RBE3/G')
m51_RBE3_IDENTITY_df = extract_df(m51_model,'/NASTRAN/INPUT/ELEMENT/RBE3/IDENTITY')
m51_RBE3_WTCG_df = extract_df(m51_model,'/NASTRAN/INPUT/ELEMENT/RBE3/WTCG')

In [30]:
print(m51_RBE2_GM_df.shape)
print(m51_RBE2_RB_df.shape)
print(m51_RBE3_G_df.shape)
print(m51_RBE3_IDENTITY_df.shape)
print(m51_RBE3_WTCG_df.shape)

(2300, 1)
(142, 7)
(606315, 1)
(118162, 9)
(580028, 4)


In [31]:
m28_RBE3_IDENTITY_df.head()

,EID,REFG,REFC,WTCG_POS,WTCG_LEN,GM_POS,GM_LEN,ALPHA,DOMAIN_ID
0,219995,219995,123456,0,1,0,0,0.0,1
1,1901008,1901008,123456,1,1,0,0,0.0,1
2,1901009,1901009,123456,2,1,0,0,0.0,1
3,1901010,1901010,123456,3,1,0,0,0.0,1
4,1901108,1901208,123456,4,1,0,0,0.0,1


In [32]:
m28_RBE3_G_df.head()

,ID
0,1124001
1,1124003
2,1124004
3,1124005
4,1124006


In [35]:
node_df = m28_model.get_node('/NASTRAN/INPUT/NODE/GRID')

In [46]:
arr = node_df.col('X')[:10,:]

In [62]:
m28_model.close()
m51_model.close()